# Clase 02 - Algoritmos en grafos

En esta actividad vamos a programar dos algoritmos importantes a la hora de analizar un grafo:

- PageRank: algoritmo que nos sirve para rankear nodos de acuerdo a su importancia
- Algoritmo de Dijkstra: algoritmo que nos permite encontrar las distancias más cortas entre nodos.

## 01 - PageRank

Como vimos en clases, PageRank es un algoritmo iterativo que le otorga una calificación a cada nodo. Mientras más alta esta calificación, más alta es la importancia de este nodo en la red. Una fórmula general para PageRank es la siguiente:

$$
PR_t(n_i) = \frac{1-d}{N} + d \sum_{n_j\ \in\ In(n_i)}\frac{PR_{t-1}(n_j)}{Out(n_j)}
$$

En la que: 

- $PR_t(n_i)$ es el PageRank del nodo $i$ en la iteración $t$.
- $N$ es el número de nodos en el grafo.
- $d$ es el _damping factor_ (usualmente 0.85).
- $In(n_i)$ es una función que retorna todos los nodos que apuntan hacia $n_i$.
- $Out(n_j)$ es una función que retorna el número de nodos hacia los que $n_j$ apunta.

### P1 - Función `pagerank`

En esta tarea deben implementar en Python una función llamada `pagerank` que recibe una matriz de adyacencia y retorna una lista con el PageRank de cada uno de los nodos.

In [1]:
from functools import reduce

def norm_difference(v1, v2):
    v_difference = [(v1[i] - v2[i]) for i in range(len(v1))]
    v_squared = map((lambda x: (x)**2), v_difference)
    return (reduce((lambda x, y: x + y), v_squared))**(1/2)

def calculate_pagerank_node(node, v, M, d, N):
    # node es el id del nodo que está siendo calculado
    # v es el vector de PageRank en la iteración pasada
    # M es la matriz de adyacencia
    pagerank_node = 0
    for j in range(N):
        if M[j][node] == 1:
            outgoing_links = reduce((lambda x, y: x + y), M[j])
            pagerank_node += v[j]/outgoing_links
    pagerank_node = (1-d)/N + d*pagerank_node
    return pagerank_node
            
    

def pagerank(M, eps=1.0e-8, d=0.85):
    N = len(M)
    v = [1/N for _ in range(N)]
    last_v = [100 for _ in range(N)]
    
    # Calculamos el PageRank de cada nodo
    while norm_difference(v, last_v) > eps:
        
        last_v = []
        for element in v:
            last_v.append(element)
        
        v = []
        for i in range(N):
            v.append(calculate_pagerank_node(i, last_v, M, d, N))
        
    return v

M = [[0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 0],
     [0, 1, 1, 0]]

# Llamamos a la función
pr = pagerank(M, 0.0001)
print(pr)

[0.037500000000000006, 0.08531250000000001, 0.125953125, 0.037500000000000006]


### P2 - Comprobar resultados

Ahora considere el siguiente grafo construido con la librería `networkx`:

In [2]:
import networkx as nx

NODES = [1, 2, 3, 4, 5, 6]

# Estas son las aristas
G = nx.DiGraph()
G.add_nodes_from(NODES)
G.add_edge(1, 3)
G.add_edge(2, 3)
G.add_edge(4, 3)
G.add_edge(2, 5)
G.add_edge(3, 5)
G.add_edge(4, 5)
G.add_edge(6, 5)

print('El PageRank del grafo es:', nx.pagerank(G))

El PageRank del grafo es: {1: 0.08550676865904419, 2: 0.08550676865904419, 3: 0.23086724332915926, 4: 0.08550676865904419, 5: 0.42710568203466376, 6: 0.08550676865904419}


Construya la matriz de adyacencia del grafo utilizando listas de Python y compute el PageRank utilizando la función que definió en el punto anterior. Compare que sus resultados sean los mismos y explique el resultado.

**Observación**: El PageRank calculado por Networkx está normalizado, por lo que debes normalizarlos para comparar los resultados.

## 02 - Algoritmo de Dijkstra

Como vimos en clases, el Algoritmo de Dijkstra nos sirve para computar caminos más cortos entre nodos en que las aristas tienen un costo asociado. Suponga que el usuario va a entregar una matriz de adyacencia que en vez de un 1 posee el costo de ir de un nodo a otro. Por ejemplo, esta sería un posible input:

```Python
M = [[0, 10, 0, 4, 3],
     [0, 0, 2, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 3, 0, 0, 0],
     [0, 0, 15, 0, 0]]
```

### P1 - Función `dijkstra_algorithm`

Haga una función llamada `dijkstra_algorithm` que reciba la matriz de adyacencia junto con el identificador de un nodo de partida. La función debe retornar cada nodo junto al costo del camino más corto para llegar a él, partiendo desde el nodo inicial.

In [23]:
from math import inf

def get_by_priority(to_explore, costs):
    minimum = inf
    min_pos = -1
    for element in to_explore:
        if costs[element][1] < minimum:
            minimum = costs[element][1]
            min_pos = element
    return min_pos

def dijkstra_algorithm(M, source):
    visited = set()
    to_explore = []
    N = len(M)
    costs = [(-1, inf) for i in range(N)]
    costs[source] = (source, 0)
    
    to_explore.append(source)
    
    while to_explore:
        min_pos = get_by_priority(to_explore, costs)
        visited.add(min_pos)
        to_explore.remove(min_pos)
        for i in range(N):
            if i not in visited and M[min_pos][i] > 0:
                if M[min_pos][i] + costs[min_pos][1] < costs[i][1]:
                    costs[i] = (min_pos, M[min_pos][i] + costs[min_pos][1])
                    if i not in to_explore:
                        to_explore.append(i)
    
    return costs

### P2 - Encontrar el camino

Use el algoritmo anterior para crear una función llamada `shortest_path` que recibe la matriz de adyacencia, un nodo inicial y un nodo objetivo y entregue los nodos por los que debo pasar para llegar desde el nodo inicial al final con menor costo. Asuma que el camino entre esos dos nodos existe.

In [36]:
def get_parent(M, s, t, l):
    if costs[t][0] == s:
        l.append(costs[t][0])
        return l
    else:
        l.append(costs[t][0])
        return get_parent(M, s, costs[t][0], l)

def shortest_path(M, s, t):
    costs = dijkstra_algorithm(M, s)
    if s == t:
        print("Son el mismo nodo")
    else:
        l = [t]
        return get_parent(M, s, t, l)
    
M = [[0, 10, 0, 4, 3],
     [0, 0, 2, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 3, 0, 0, 0],
     [0, 0, 15, 0, 0]]

s = 0
t = 2
l = shortest_path(M, s, t)

for i in range(len(l)):
    print("Nodo", l[len(l) - 1 - i])

Nodo 0
Nodo 3
Nodo 1
Nodo 2
